In [1]:
import os
import pandas as pd
import re
import numpy as np
import tensorflow as tf

2024-02-09 22:24:46.184376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from OneHotEncoding import get_one_hot_sequence, one_hot
from BuildMatrixFromDotBracket import get_couples, build_matrix

In [3]:
# getting input

# defining folder path
bprna_folder_path = "../../DATASETS/bpRNA_1m_90/bpRNA_1m_90_DBNFILES_as_TXT/"
pattern = r"bpRNA_CRW_.*\.txt$" # get files that start with "bpRNA_CRW_"
extracted_data = {}
bprna_df = pd.DataFrame(columns=["name", "length", "sequence", "dbn"])

# iterate through 
for filename in os.listdir(bprna_folder_path):
    if re.match(pattern, filename):
        with open(os.path.join(bprna_folder_path, filename), "r") as file:
            for i, line in enumerate(file):
                if i == 0:
                    key, value = line.strip().split(" ", 1)
                    extracted_data["name"] = value
                elif i == 1:
                    key, value = line.strip().split(" ", 1)
                    extracted_data["length"] = value
                elif i == 3:
                    extracted_data["sequence"] = line
                elif i == 4:
                    extracted_data["dbn"] = line
        bprna_df = pd.concat([bprna_df, pd.DataFrame([extracted_data])], ignore_index=True)
        extracted_data = {}

In [4]:
example_structure = bprna_df.loc[8, "dbn"]
example_sequence_length = int(bprna_df.loc[8, "length"])
example_couples = get_couples(example_structure)
example_matrix = build_matrix(example_couples, example_sequence_length)
# example_one_hot = get_one_hot_sequence(bprna_df.loc[8, "sequence"])
example_one_hot = one_hot(bprna_df.loc[8, "sequence"])
example_one_hot = example_one_hot[:-1, :]

In [5]:
example_2_structure = "...(((((((..((((((.........))))))......).((((((.......))))))..))))))..."
example_2_sequence = "CGCUUCAUAUAAUCCUAAUGAUAUGGUUUGGGAGUUUCUACCAAGAGCCUUAAACUCUUGAUUAUGAAGUG"
example_2_length = len(example_2_sequence)

example_2_one_hot = one_hot(example_2_sequence)
example_2_matrix = build_matrix(get_couples(example_2_structure), example_2_length)

In [6]:
print(example_sequence_length)
print(example_one_hot)
print(len(example_one_hot))

370
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 ...
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]]
370


In [7]:
A = get_one_hot_sequence("A")
U = get_one_hot_sequence("U")
G = get_one_hot_sequence("G")
C = get_one_hot_sequence("C")
allowed_pairs = [(A, U), (U, A), (G, C), (C, G), (G, U), (U, G)]

In [72]:
example_2_one_hot.shape

(71, 4)

In [110]:
input_layer = tf.keras.layers.Input(shape=(71, 4))
# input_layer = tf.keras.layers.Input(shape=(None, 4))

In [111]:
lstm_layer = tf.keras.layers.LSTM(units=64, return_sequences=True)(input_layer)

In [112]:
flattened = tf.keras.layers.Flatten()(lstm_layer)

In [113]:
global_max_pooled = tf.keras.layers.GlobalMaxPooling1D()(lstm_layer)

In [114]:
output_layer = tf.keras.layers.Dense(example_2_length * example_2_length, activation='sigmoid')(flattened)
# output_layer = tf.keras.layers.Dense(example_2_length * example_2_length, activation='sigmoid')(global_max_pooled)

In [116]:
adjacency_matrix = tf.reshape(output_layer, (-1, example_2_length, example_2_length))

In [117]:
model = tf.keras.Model(inputs=input_layer, outputs=adjacency_matrix)

In [118]:
# # Input layer for variable-length sequences of one-hot encodings
# input_layer = tf.keras.layers.Input(shape=(example_sequence_length, 4))

# # Recurrent layer to process sequential information
# lstm_layer = tf.keras.layers.LSTM(units=64, return_sequences=True)(input_layer)

# flattened = tf.keras.layers.Flatten()(lstm_layer)

# # Output layer for adjacency matrix
# output_layer = tf.keras.layers.Dense(example_sequence_length * example_sequence_length, activation='sigmoid')(flattened)

# # Reshape into adjacency matrix
# adjacency_matrix = tf.reshape(output_layer, (-1, example_sequence_length, example_sequence_length))

# # Build the model
# model = tf.keras.Model(inputs=input_layer, outputs=adjacency_matrix)

In [119]:
# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [120]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(1, 71, 4)]              0         
                                                                 
 lstm_13 (LSTM)              (1, 71, 64)               17664     
                                                                 
 flatten_9 (Flatten)         (1, 4544)                 0         
                                                                 
 dense_10 (Dense)            (1, 5041)                 22911345  
                                                                 
 tf.reshape_10 (TFOpLambda)  (1, 71, 71)               0         
                                                                 
Total params: 22929009 (87.47 MB)
Trainable params: 22929009 (87.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [121]:
# X = np.expand_dims(np.arange(0, example_2_length, 4), axis=1)
# y = np.expand_dimsnp.arange((0, example_2_length, 4), axis=1)
# X = tf.stack(example_2_one_hot)
# y = tf.stack(example_2_matrix)
X = np.array(example_2_one_hot)
y = np.array(example_2_matrix)

In [123]:
example_2_one_hot = example_2_one_hot.reshape(-1, 71, 4)

In [124]:
model.fit(example_2_one_hot, example_2_matrix, epochs=10)
# model.fit(X, y, epochs=10)

ValueError: Data cardinality is ambiguous:
  x sizes: 1
  y sizes: 71
Make sure all arrays contain the same number of samples.